In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser, StrOutputParser
import boto3
import os
import re
import json
from datetime import datetime
from pydantic import BaseModel
from time_converter import TimeConverter
import kss
load_dotenv()

True

In [2]:
subject = '수학'
room_id = '674e58163925f28f6caf4fa0'

s3 = boto3.client('s3')

bucket_name = 'pagecall-text'
parent_folder = f'{subject}/{room_id}'

def get_items(bucket, folder):
    response = s3.list_objects_v2(Bucket=bucket, Prefix=folder)
    content_list = []
    for content in response.get('Contents', []):
        key = content.get('Key', None)
        if len(key.split('/')) == 4:
            content_list.append(key)
    return content_list

def download_items(bucket, file_keys, local_dir):
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
    
    for file_key in file_keys:
        folder_name = os.path.basename(os.path.dirname(file_key))
        file_name = os.path.basename(file_key)
        local_file_path = os.path.join(local_dir, f"{folder_name}_{file_name}")
        s3.download_file(bucket, file_key, local_file_path)
        
file_keys = get_items(bucket_name, parent_folder)
download_items(bucket_name, file_keys, 'downloads')

In [3]:
local_download_dir = './downloads'

def merge_files(directory):
    merged_data = []
    
    for file_name in os.listdir(directory):
        parts = file_name.split('_')
        absolute_start_time = int(parts[-2])
        speaker = 'teacher' if 'T' in parts[0] else 'student'
    
        file_path = os.path.join(directory, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = json.load(f)
            
            for item in content:
                start_ms = TimeConverter.convert_time_to_milliseconds(item['start'])
                end_ms = TimeConverter.convert_time_to_milliseconds(item['end'])
                
                merged_item = {
                    "absolute_start_time": absolute_start_time + start_ms,
                    "absolute_end_time": absolute_start_time + end_ms,
                    "time_range": None,
                    "speaker": speaker,
                    "text": item["text"].strip(),
                }
                merged_data.append(merged_item)
                
    # 절대 시작 시간을 기준으로 정렬
    merged_data.sort(key=lambda x: x["absolute_start_time"])

    global_start_time = merged_data[0]["absolute_start_time"]  # 가장 이른 절대 시작 시간
    for item in merged_data:
        item["time_range"] = f"{TimeConverter.format_ms_to_xm_ys(item['absolute_start_time'] - global_start_time)} ~ {TimeConverter.format_ms_to_xm_ys(item['absolute_end_time'] - global_start_time)}"
        item['absolute_start_time'] = TimeConverter.format_ms_to_ktc(item['absolute_start_time'])
        item['absolute_end_time'] = TimeConverter.format_ms_to_ktc(item['absolute_end_time'])
    
    return merged_data

In [4]:
raw_data = merge_files(local_download_dir)

def extract_data(is_time, speaker=None):
    if (is_time) & (speaker == None):
        return [{'time_range': item['time_range'], 'speaker': item['speaker'], 'text': item['text']} for item in raw_data]
    if (is_time) & (speaker == 'teacher'):
        return [{'time_range': item['time_range'], 'speaker': item['speaker'], 'text': item['text']} for item in raw_data if item['speaker'] == 'teacher']
    if (is_time) & (speaker == 'student'):
        return [{'time_range': item['time_range'], 'speaker': item['speaker'], 'text': item['text']} for item in raw_data if item['speaker'] == 'student']
    if (is_time==False) & (speaker == 'teacher'):
        return ' '.join([item['text'] for item in raw_data if item['speaker'] == 'teacher'])
    if (is_time==False) & (speaker == 'student'):
        return ' '.join([item['text'] for item in raw_data if item['speaker'] == 'student'])

extracted_data = extract_data(is_time=False, speaker='teacher')
extracted_data

'목소리 잘 들릴까요? 안녕하세요. 잘 들리고 그러면 필기 한번 확인해 볼게요. 잘 보여요? 잠깐만요. 혹시 전에 선택으로 과외 해본 적 있어요? 아 정말요? 저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요? 이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요 아 그거 괜찮아요 같이 하면 되니까 이게 왜 두 개였냐면 이게 상반기용이 하나 있었고, 하반기용이 하나 있었거든요. 그래서 이 파트가 지금 하반기용이고, 이 도형이 많은 게 상반기용이더라고요. 그래서 일단은 지금 하반기니까, 하반기 거를 일단 하고, 기말고사 얼마 안 남았으니까 샌쩍 같이 본 다음에 시간이 남으면 상반기 걸 같이 볼게요 그리고 제가 이거 답을 한번 봤는데 일단 풍거는 거의 다 맞았던 것 같아요 네 그래서 풍거는 잘 푸는 것 같고 별표 위주로 한번 보면 될 것 같고 그 다음에 이게 지금 교재가 깨끗한 상태잖아요. 근데 하윤 학생이 뭘 틀렸는지 제가 모르니까. 이거를 하윤 학생이 그때그때 말해주면 좋을 것 같아요. 이 시간이 왜 지금 가고 있는거지? 지금 시작할게요. 자기소개부터 해볼게요. 제 이름은 이지민이고 저는 지금 연세대학교 산업공학과에 지금 재학중이에요. 다음이 과외 경험. 과외 경험. 이게 제가 3수로 해서 이 대학교를 갔거든요. 그래가지고 과외는 상반기가 한 6월? 내해 6월 전까지만 진행하다가 이제 수능 공부하고 그랬어가지고 과외를 일단 고3 학생 생명... 아이고야. 생명이랑 지구를 한 5개월 정도 했던 것 같고 친구가 성적이 거의 노베였는데 육모까지만 봐주고 제가 그만뒀거든요. 그 수능 준비를 제가 해야 돼가지고 3,4등급 나왔던 것 같아요. 근데 지구가 녹였고 생명은 약간 베이스가 있는 친구였어요. 공부한 적이 있는 친구였고. 근데 이 친구가 숙제를 제대로 안 써가지고 숙제가 제대로 했어도 더 올랐을 텐데 참 아쉽습니다. 어쨌든 다음이 0,3 친구. 그렇게 3학년 친구 수학을 했었고 이 친구도 한 5, 6개월 했던 것 같아요. 주로 제가 겨울에 시작

In [5]:
splited_data = []
idx = 0
for sent in kss.split_sentences(extracted_data):
    if len(sent) > 5:
        splited_data.append({'idx': idx, 'text': sent})
        idx += 1
splited_data

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



[{'idx': 0, 'text': '목소리 잘 들릴까요?'},
 {'idx': 1, 'text': '안녕하세요.'},
 {'idx': 2, 'text': '잘 들리고 그러면 필기 한번 확인해 볼게요.'},
 {'idx': 3, 'text': '잘 보여요?'},
 {'idx': 4, 'text': '혹시 전에 선택으로 과외 해본 적 있어요?'},
 {'idx': 5, 'text': '아 정말요?'},
 {'idx': 6, 'text': '저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요?'},
 {'idx': 7, 'text': '이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요'},
 {'idx': 8, 'text': '아 그거 괜찮아요'},
 {'idx': 9, 'text': '같이 하면 되니까 이게 왜 두 개였냐면 이게 상반기용이 하나 있었고, 하반기용이 하나 있었거든요.'},
 {'idx': 10, 'text': '그래서 이 파트가 지금 하반기용이고, 이 도형이 많은 게 상반기용이더라고요.'},
 {'idx': 11,
  'text': '그래서 일단은 지금 하반기니까, 하반기 거를 일단 하고, 기말고사 얼마 안 남았으니까 샌쩍 같이 본 다음에 시간이 남으면 상반기 걸 같이 볼게요'},
 {'idx': 12, 'text': '그리고 제가 이거 답을 한번 봤는데 일단 풍거는 거의 다 맞았던 것 같아요'},
 {'idx': 13,
  'text': '네 그래서 풍거는 잘 푸는 것 같고 별표 위주로 한번 보면 될 것 같고 그 다음에 이게 지금 교재가 깨끗한 상태잖아요.'},
 {'idx': 14, 'text': '근데 하윤 학생이 뭘 틀렸는지 제가 모르니까.'},
 {'idx': 15, 'text': '이거를 하윤 학생이 그때그때 말해주면 좋을 것 같아요.'},
 {'idx': 16, 'text': '이 시간이 왜 지금 가고 있는거지?'},
 {'idx': 17, 'text': '지금 시작할게요.'},
 {'idx': 18, 'text'

In [6]:
def split_with_overlap(data, chunk_size, overlap):
    result = []
    step = chunk_size - overlap
    for i in range(0, len(data), step):
        result.append(data[i:i + chunk_size])
        if i + chunk_size >= len(data):
            break
    return result

chunk_size = 30
overlap = 5
chunks_with_overlap = split_with_overlap(splited_data, chunk_size, overlap)

In [7]:
from langchain_core.runnables import RunnableParallel
from prompt.question_analyzer import QuestionAnalyzer

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key='sk-602a9dbe731d48b0afab59d3c140c583', 
    openai_api_base='https://api.deepseek.com',
)

system_prompt = QuestionAnalyzer(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
question_analysis_results = chain.batch([
    {"user_message": chunk} for chunk in chunks_with_overlap
])

In [8]:
def extract_indices(data):
    all_indices = []
    for entry in data:
        json_text = re.search(r'```json\n(.*?)\n```', entry, re.DOTALL).group(1)
        parsed_json = json.loads(json_text)
        all_indices.extend(parsed_json[0]["idx"])
    return sorted(set(all_indices))

question_indices = extract_indices(question_analysis_results)
question_indices

[0,
 3,
 4,
 6,
 16,
 46,
 48,
 75,
 81,
 86,
 98,
 104,
 105,
 108,
 110,
 117,
 121,
 124,
 126,
 130,
 136,
 137,
 141,
 150,
 152,
 158,
 159,
 166,
 169,
 171,
 177,
 178,
 182,
 187,
 188,
 189,
 192,
 193,
 195,
 197,
 199,
 203,
 210,
 215,
 217,
 225,
 226,
 227,
 229,
 231,
 232,
 233,
 235,
 239,
 240,
 250,
 251,
 257,
 264,
 270,
 299,
 303,
 305,
 311,
 312,
 314,
 316,
 318,
 322,
 326,
 345,
 352,
 353,
 356,
 362,
 363,
 365,
 367,
 368,
 369,
 371,
 380,
 382,
 385,
 386,
 387,
 388,
 394,
 397,
 401,
 402,
 403,
 410,
 412,
 416,
 417,
 418,
 420,
 425,
 427,
 433,
 434,
 435,
 436,
 444,
 448,
 450,
 452,
 460,
 461,
 463,
 467,
 468,
 475,
 476,
 477,
 486,
 493,
 497,
 500,
 504,
 508,
 511,
 515,
 517,
 521,
 523,
 524,
 527,
 530,
 531,
 533,
 534,
 535,
 536,
 537,
 538,
 539,
 541,
 542,
 543,
 544,
 551,
 553,
 554,
 559,
 579,
 589,
 591,
 596,
 597,
 605,
 610]

In [9]:
[item for item in splited_data if item['idx'] in question_indices]

[{'idx': 0, 'text': '목소리 잘 들릴까요?'},
 {'idx': 3, 'text': '잘 보여요?'},
 {'idx': 4, 'text': '혹시 전에 선택으로 과외 해본 적 있어요?'},
 {'idx': 6, 'text': '저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요?'},
 {'idx': 16, 'text': '이 시간이 왜 지금 가고 있는거지?'},
 {'idx': 46, 'text': '이건 뭘 말해주면 좋을까..'},
 {'idx': 48, 'text': '제가 혹시 화성고 알아요?'},
 {'idx': 75,
  'text': '오고 잘 딱 네 그리고 음 네 네 네 네 네 네 음 음 그쵸가 주차 없으니까 음 그 다음 네 네 네. 네. 아하. 네. 네. 네. 네. 네. 으 으 으 으 으 으 으 으 으 으 으 으 으 으 으 으 어떤 거부터 시작한다는 게 정확히 무슨 말이에요?'},
 {'idx': 81, 'text': '나 아직 중학생이니까 괜찮아요?'},
 {'idx': 86, 'text': '요 한달 계획도?'},
 {'idx': 98, 'text': '이것도 감으로 본 거예요?'},
 {'idx': 104, 'text': '이런게 약간 빵꾸가 있구나 1차음수 이런거 얘기하는거에요?'},
 {'idx': 105, 'text': '1차강정식, 부등식 요쪽?'},
 {'idx': 108, 'text': '그러면 일단 4번부터 볼까요?'},
 {'idx': 110, 'text': '해가 없을 조건이 해가 없어야 되는 조건의 조건을 알아요?'},
 {'idx': 117, 'text': '그럼 위에 거보다 정리를 해주면 OX 플러스 2Y는 6이고 밑에 거 한번 정리해볼까요?'},
 {'idx': 121, 'text': '요렇게 되죠?'},
 {'idx': 124, 'text': '태윤 학생 안들려요?'},
 {'idx': 126, 'text': '어, 들리죠?'},
 {'idx': 130, 'text': '이런 연

In [10]:
def get_grouped_surrounding_texts(data, target_indices, range_size=5):
    grouped_result = []
    try:
        for idx in target_indices:
            question = data[idx]['text']
            start_idx = max(0, idx - range_size)
            end_idx = min(len(data), idx + range_size + 1)
            contexts = data[start_idx:end_idx]
            context = ' '.join([item['text'] for item in contexts])
            grouped_result.append({'idx': idx, 'question': question, 'context': context})
        return grouped_result
    except:
        print(idx)

# 결과 출력
grouped_surrounding_texts = get_grouped_surrounding_texts(splited_data, question_indices)
grouped_surrounding_texts

[{'idx': 0,
  'question': '목소리 잘 들릴까요?',
  'context': '목소리 잘 들릴까요? 안녕하세요. 잘 들리고 그러면 필기 한번 확인해 볼게요. 잘 보여요? 혹시 전에 선택으로 과외 해본 적 있어요? 아 정말요?'},
 {'idx': 3,
  'question': '잘 보여요?',
  'context': '목소리 잘 들릴까요? 안녕하세요. 잘 들리고 그러면 필기 한번 확인해 볼게요. 잘 보여요? 혹시 전에 선택으로 과외 해본 적 있어요? 아 정말요? 저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요? 이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요 아 그거 괜찮아요'},
 {'idx': 4,
  'question': '혹시 전에 선택으로 과외 해본 적 있어요?',
  'context': '목소리 잘 들릴까요? 안녕하세요. 잘 들리고 그러면 필기 한번 확인해 볼게요. 잘 보여요? 혹시 전에 선택으로 과외 해본 적 있어요? 아 정말요? 저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요? 이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요 아 그거 괜찮아요 같이 하면 되니까 이게 왜 두 개였냐면 이게 상반기용이 하나 있었고, 하반기용이 하나 있었거든요.'},
 {'idx': 6,
  'question': '저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요?',
  'context': '안녕하세요. 잘 들리고 그러면 필기 한번 확인해 볼게요. 잘 보여요? 혹시 전에 선택으로 과외 해본 적 있어요? 아 정말요? 저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요? 이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요 아 그거 괜찮아요 같이 하면 되니까 이게 왜 두 개였냐면 이게 상반기용이 하나 있었고, 하반기용이 하나 있었거든요. 그래서 이 파트가 지금 하반기용이고, 이 도형이 많은 게 상반기용이더라고요. 그래서 일단은 지금 하반기니까, 하반기 거를 일

In [11]:
from langchain_core.runnables import RunnableParallel
from prompt.question_classifier import QuestionClassifier

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key='sk-602a9dbe731d48b0afab59d3c140c583', 
    openai_api_base='https://api.deepseek.com',
)

system_prompt = QuestionClassifier(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

question_classifier_results = chain.batch([
    {"user_message": chunk} for chunk in grouped_surrounding_texts
])

In [12]:
question_classifier_results

['False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'False',
 'False',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'False',
 'False',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'False',
 'True',
 'False',
 'False',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'False',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 '

In [13]:
for (res,item) in zip(question_classifier_results, grouped_surrounding_texts):
    item['result'] = res
grouped_surrounding_texts

[{'idx': 0,
  'question': '목소리 잘 들릴까요?',
  'context': '목소리 잘 들릴까요? 안녕하세요. 잘 들리고 그러면 필기 한번 확인해 볼게요. 잘 보여요? 혹시 전에 선택으로 과외 해본 적 있어요? 아 정말요?',
  'result': 'False'},
 {'idx': 3,
  'question': '잘 보여요?',
  'context': '목소리 잘 들릴까요? 안녕하세요. 잘 들리고 그러면 필기 한번 확인해 볼게요. 잘 보여요? 혹시 전에 선택으로 과외 해본 적 있어요? 아 정말요? 저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요? 이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요 아 그거 괜찮아요',
  'result': 'False'},
 {'idx': 4,
  'question': '혹시 전에 선택으로 과외 해본 적 있어요?',
  'context': '목소리 잘 들릴까요? 안녕하세요. 잘 들리고 그러면 필기 한번 확인해 볼게요. 잘 보여요? 혹시 전에 선택으로 과외 해본 적 있어요? 아 정말요? 저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요? 이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요 아 그거 괜찮아요 같이 하면 되니까 이게 왜 두 개였냐면 이게 상반기용이 하나 있었고, 하반기용이 하나 있었거든요.',
  'result': 'False'},
 {'idx': 6,
  'question': '저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요?',
  'context': '안녕하세요. 잘 들리고 그러면 필기 한번 확인해 볼게요. 잘 보여요? 혹시 전에 선택으로 과외 해본 적 있어요? 아 정말요? 저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요? 이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요 아 그거 괜찮아요 같이 하면 되니까 이게 왜 두 개였냐면 이게 상반기용이 하나 있었고, 하반기용이 하나 있었거든요. 그래서 이 

In [14]:
final_feed = [{'idx': item['idx'], 'question': item['question'], 'context': item['context']} for item in grouped_surrounding_texts if item['result'] == 'True']
final_feed

[{'idx': 104,
  'question': '이런게 약간 빵꾸가 있구나 1차음수 이런거 얘기하는거에요?',
  'context': '일단 그러면 일단 얘는 거 체크표시 해놓고 갈게요. 그냥 그냥 그냥 틀렸다는 표시를 할게요. 통일하면 좋을 것 같아요. 5번에 2번. 이거 끝에 두개는 약간 그냥 날린거에요 이런게 약간 빵꾸가 있구나 1차음수 이런거 얘기하는거에요? 1차강정식, 부등식 요쪽? 네 아 알겠어요. 그럼 나오면 그때그때 개념 정리해주면 되겠다. 그러면 일단 4번부터 볼까요? 연립방정식 연립방정식 얘랑 얘의 해가 없을 조건을 구해달래요.'},
 {'idx': 105,
  'question': '1차강정식, 부등식 요쪽?',
  'context': '그냥 그냥 그냥 틀렸다는 표시를 할게요. 통일하면 좋을 것 같아요. 5번에 2번. 이거 끝에 두개는 약간 그냥 날린거에요 이런게 약간 빵꾸가 있구나 1차음수 이런거 얘기하는거에요? 1차강정식, 부등식 요쪽? 네 아 알겠어요. 그럼 나오면 그때그때 개념 정리해주면 되겠다. 그러면 일단 4번부터 볼까요? 연립방정식 연립방정식 얘랑 얘의 해가 없을 조건을 구해달래요. 해가 없을 조건이 해가 없어야 되는 조건의 조건을 알아요?'},
 {'idx': 108,
  'question': '그러면 일단 4번부터 볼까요?',
  'context': '이거 끝에 두개는 약간 그냥 날린거에요 이런게 약간 빵꾸가 있구나 1차음수 이런거 얘기하는거에요? 1차강정식, 부등식 요쪽? 네 아 알겠어요. 그럼 나오면 그때그때 개념 정리해주면 되겠다. 그러면 일단 4번부터 볼까요? 연립방정식 연립방정식 얘랑 얘의 해가 없을 조건을 구해달래요. 해가 없을 조건이 해가 없어야 되는 조건의 조건을 알아요? 어우 끊겼다. 모르겠어요. 그러면 일단 해가 없을 조건.'},
 {'idx': 110,
  'question': '해가 없을 조건이 해가 없어야 되는 조건의 조건을 알아요?',
  'context': '1차강정식, 부등식 요쪽? 네 아 알

In [15]:
from langchain_core.runnables import RunnableParallel
from prompt import digging
import importlib

llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key='sk-602a9dbe731d48b0afab59d3c140c583', 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(digging)
system_prompt = digging.Digging(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
results = chain.batch([
    {"user_message": chunk} for chunk in final_feed
])

In [16]:
results = [json.loads(item.strip('```json').strip('```').strip()) for item in results]
results

[{'idx': 104,
  'result': 'False',
  'reason': '해당 질문은 단순히 이해하고 있는지 확인하기 위한 질문으로, 학생의 사고 과정을 묻거나 자신의 이해를 설명하도록 유도하는 메타인지형 질문이 아닙니다.'},
 {'idx': 105,
  'result': 'False',
  'reason': '해당 질문은 단순히 특정 개념에 대해 언급하고 있는지 확인하기 위한 질문으로, 학생의 사고 과정을 묻거나 메타인지를 유도하는 질문이 아닙니다.'},
 {'idx': 108,
  'result': 'False',
  'reason': '해당 질문은 단순히 다음 문제로 넘어가자는 제안으로, 학생의 생각을 묻거나 이해도를 확인하는 메타인지형 질문이 아닙니다.'},
 {'idx': 110,
  'result': 'True',
  'reason': "해당 질문은 학생에게 '해가 없을 조건'에 대해 스스로 생각하고 이해하고 있는지 확인하는 메타인지형 질문입니다. 학생이 해당 개념을 알고 있는지, 그리고 왜 그런 조건이 필요한지 설명하도록 유도하고 있습니다."},
 {'idx': 117,
  'result': 'False',
  'reason': '해당 질문은 학생에게 단순히 식을 정리하도록 요구하는 것으로, 학생의 사고 과정이나 이해도를 탐구하는 메타인지형 질문이 아닙니다.'},
 {'idx': 121,
  'result': 'False',
  'reason': '해당 질문은 단순히 확인을 위한 질문으로, 학생의 생각을 묻거나 이해도를 파악하기 위한 메타인지형 질문이 아닙니다.'},
 {'idx': 130,
  'result': 'True',
  'reason': '해당 질문은 학생에게 연립방정식에서 해가 없을 조건에 대해 생각하게 하고, 이를 일반화하여 설명하도록 유도하는 메타인지형 질문입니다.'},
 {'idx': 136,
  'result': 'False',
  'reason': '해당 질문은 단순히 개념을 적용했을 때의 결과를 묻는 것으로, 

In [17]:
for (res, item) in zip(results, final_feed):
    item['result'] = res['result']
    item['reason'] = res['reason']
final_feed

[{'idx': 104,
  'question': '이런게 약간 빵꾸가 있구나 1차음수 이런거 얘기하는거에요?',
  'context': '일단 그러면 일단 얘는 거 체크표시 해놓고 갈게요. 그냥 그냥 그냥 틀렸다는 표시를 할게요. 통일하면 좋을 것 같아요. 5번에 2번. 이거 끝에 두개는 약간 그냥 날린거에요 이런게 약간 빵꾸가 있구나 1차음수 이런거 얘기하는거에요? 1차강정식, 부등식 요쪽? 네 아 알겠어요. 그럼 나오면 그때그때 개념 정리해주면 되겠다. 그러면 일단 4번부터 볼까요? 연립방정식 연립방정식 얘랑 얘의 해가 없을 조건을 구해달래요.',
  'result': 'False',
  'reason': '해당 질문은 단순히 이해하고 있는지 확인하기 위한 질문으로, 학생의 사고 과정을 묻거나 자신의 이해를 설명하도록 유도하는 메타인지형 질문이 아닙니다.'},
 {'idx': 105,
  'question': '1차강정식, 부등식 요쪽?',
  'context': '그냥 그냥 그냥 틀렸다는 표시를 할게요. 통일하면 좋을 것 같아요. 5번에 2번. 이거 끝에 두개는 약간 그냥 날린거에요 이런게 약간 빵꾸가 있구나 1차음수 이런거 얘기하는거에요? 1차강정식, 부등식 요쪽? 네 아 알겠어요. 그럼 나오면 그때그때 개념 정리해주면 되겠다. 그러면 일단 4번부터 볼까요? 연립방정식 연립방정식 얘랑 얘의 해가 없을 조건을 구해달래요. 해가 없을 조건이 해가 없어야 되는 조건의 조건을 알아요?',
  'result': 'False',
  'reason': '해당 질문은 단순히 특정 개념에 대해 언급하고 있는지 확인하기 위한 질문으로, 학생의 사고 과정을 묻거나 메타인지를 유도하는 질문이 아닙니다.'},
 {'idx': 108,
  'question': '그러면 일단 4번부터 볼까요?',
  'context': '이거 끝에 두개는 약간 그냥 날린거에요 이런게 약간 빵꾸가 있구나 1차음수 이런거 얘기하는거에요? 1차강정식, 부등식 요쪽? 네 아 알겠어요. 그럼 나오면 그

In [18]:
with open("이지민T-양하윤S.txt", "w", encoding="utf-8-sig") as file:
    json.dump(final_feed, file, ensure_ascii=False, indent=4)